In [1]:
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from dotenv import load_dotenv
from datetime import datetime, timedelta
import csv

In [2]:
# Función para ajustar la fecha
def ajustar_fecha(fecha):
    current_year = datetime.now().year
    parts = fecha.split('-')
    
    if len(parts) == 3:
        year, month, day = parts
        return f"{day}-{month}-{year}"
    elif len(parts) == 2:
        month, day = parts
        return f"{day}-{month}-{current_year}"
    elif "Hace" in fecha:
        # Convertir el texto "Hace X día(s)" en una fecha
        cantidad_dias = int(fecha.split()[1])
        fecha_hace_dias = datetime.now() - timedelta(days=cantidad_dias)
        return fecha_hace_dias.strftime("%d-%m-%Y")
    else:
        return fecha  # Devuelve la fecha original si no se puede ajustar

# Función para formatear las vistas (views) en números completos
def formatear_vistas(vistas):
    if vistas.endswith('K'):
        views_number = float(vistas[:-1]) * 1000
    elif vistas.endswith('M'):
        views_number = float(vistas[:-1]) * 1000000
    else:
        views_number = float(vistas)
    return str(int(views_number))


In [3]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la ruta al Chrome WebDriver desde la variable de entorno
chrome_driver_path = os.getenv('config_chromedrive')

In [4]:
service = Service(executable_path=chrome_driver_path)

# Configurar las opciones del navegador
options = webdriver.ChromeOptions()
options.headless = False  # Cambia a True si deseas que el navegador sea invisible

# Inicializar el WebDriver
driver = webdriver.Chrome(service=service, options=options)

# URL de la página web a scrapear
urlI = os.getenv('url_mansaI')
urlII = os.getenv('url_mansaII')

# Navegar a la URL
driver.get(urlII)

time.sleep(15)  # Esperar a que se cargue la página

scrolls = 4

for _ in range(scrolls):
    # Ejecuta JavaScript para hacer scroll hacia abajo en la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Espera un breve momento para que el contenido se cargue después del scroll
    time.sleep(15)
    
time.sleep(15)

In [5]:
container_xpath = '//div[contains(@class, "css-1soki6-DivItemContainerForSearch") and contains(@class, "e19c29qe10")]'

try:
    # Encontrar todos los elementos dentro del contenedor principal
    elements = driver.find_elements(by='xpath', value=container_xpath)
    
    # Crear un archivo CSV llamado 'bahia_mansa.csv'
    with open('Scrap/Data/bahia_mansa.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        
        # Escribir encabezados en el archivo CSV
        writer.writerow(['Fecha', 'Usuario', 'Descripción', 'Vistas', 'Hashtags'])
        
        # Iterar sobre cada elemento dentro del contenedor
        for element in elements:
                try:
                    # Extraer los datos específicos dentro de cada elemento
                    fecha = element.find_element(by='xpath', value='.//div[contains(@class, "css-dennn6-DivTimeTag") and contains(@class, "e19c29qe15")]').text
                    usuario = element.find_element(by='xpath', value='.//p[contains(@class, "css-2zn17v-PUniqueId") and contains(@class, "etrd4pu6")]').text
                    descripcion = element.find_element(by='xpath', value='.//div[contains(@class, "css-1iy6zew-DivContainer") and contains(@class, "ejg0rhn0")]').text
                    vistas = element.find_element(by='xpath', value='.//strong[contains(@class, "css-ws4x78-StrongVideoCount") and contains(@class, "etrd4pu10")]').text
                    
                    # Extraer los hashtags
                    hashtag_elements = element.find_elements(by='xpath', value='.//strong[contains(@class, "css-1p6dp51-StrongText") and contains(@class, "ejg0rhn2")]')
                    hashtags = [hashtag_element.text for hashtag_element in hashtag_elements]
                    hashtags_str = ' '.join(hashtags)
                    
                    # Ajustar la fecha y formatear las vistas
                    fecha = ajustar_fecha(fecha)
                    vistas = formatear_vistas(vistas)
                    
                    # Escribir la fila en el archivo CSV
                    writer.writerow([fecha, usuario, descripcion, vistas, hashtags_str])
                    
                    # Mostrar información en la consola
                    print(f"Fecha: {fecha}, Usuario: {usuario}, Descripción: {descripcion}, Vistas: {vistas}, Hashtags: {hashtags_str}")
                except Exception as e:
                    print("Error al extraer datos:", e)
except Exception as e:
    print("Error:", e)

finally:
    # Cerrar el navegador al finalizar
    driver.quit()

Fecha: 26-8-2022, Usuario: florscovenna_, Descripción: 🗺BAHIA MANSA en VLA📍 es soñada🤩 #villalaangostura #patagoniaargentina, Vistas: 27900, Hashtags: #villalaangostura #patagoniaargentina   
Fecha: 16-12-2023, Usuario: thiagof.okk, Descripción: Bahia manza, villa la angostura 🤍 #villalaangostura #viral #parati #pyf, Vistas: 1276, Hashtags: #villalaangostura #viral #parati #pyf       
Fecha: 29-2-2024, Usuario: aby.avg, Descripción: Apreciando ando ✨🤍#patagoniaargentina #villalaangostura #bahiamansa, Vistas: 860, Hashtags: #patagoniaargentina #villalaangostura #bahiamansa
Fecha: 21-8-2022, Usuario: luchicasim, Descripción: 🤩😮 La Casa más cara de Villa La Angostura📍#venta #inmobiliaria #argentina #realstate, Vistas: 51500, Hashtags: #venta #inmobiliaria #argentina #realstate     
Fecha: 5-1-2024, Usuario: karladf6, Descripción: Bahía Mansa Villa Angostura 📍 Argentina 🫶✨ #bahiamansavillalaangostura, Vistas: 902, Hashtags: #bahiamansavillalaangostura
Fecha: 11-9-2023, Usuario: vivamoslapa